# Reflection Pattern

The first pattern we are going to implement is the **reflection pattern**. 

---

<img src="../img/reflection_pattern.png" alt="Alt text" width="600"/>

---

This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...

**Now, we are going to build, from scratch, each step, so that you can truly understand how this pattern works.**

## Generation Step

The first thing we need to consider is:

> What do we want to generate? A poem? An essay? Python code?

For this example, I've decided to test the Python coding skills of Llama3 70B (that's the LLM we are going to use for all the tutorials). In particular, we are going to ask our LLM to code a famous sorting algorithm: **Merge Sort**. 

---

<img src="../img/mergesort.png" alt="Alt text" width="500"/>

### Groq Client and relevant imports

In [1]:
import os
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

client = Groq(api_key=os.getenv("GROQ_API_KEY"))

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python 
programmer eager to receive feedback / critique by the user.

In [2]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique," 
        "respond with a revised version of your previous attempt."
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [3]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of the Merge Sort algorithm"
    }
)

Let's generate the first version of the essay.

In [4]:
mergesort_code = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": mergesort_code
    }
)

In [5]:
display_markdown(mergesort_code, raw=True)

Here is a Python implementation of the Merge Sort algorithm:
```
def merge_sort(arr):
    """
    Sorts an array of integers using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The array to be sorted
    :return: The sorted array
    """
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left = merge_sort(left)
    right = merge_sort(right)

    return merge(left, right)


def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    result = []
    while len(left) > 0 and len(right) > 0:
        if left[0] <= right[0]:
            result.append(left.pop(0))
        else:
            result.append(right.pop(0))
    result.extend(left)
    result.extend(right)
    return result
```
Here's an explanation of how the code works:

1. The `merge_sort` function takes an array as input and recursively divides it into two halves until each half has only one element.
2. The `merge` function takes two sorted arrays as input and merges them into a single sorted array.
3. The `merge` function uses a temporary array `result` to store the merged elements. It compares the smallest element from each array and adds it to the `result` array. If one array is empty, it adds the remaining elements from the other array to the `result` array.
4. The `merge_sort` function calls itself recursively on each half of the array, and then merges the two sorted halves using the `merge` function.

Example usage:
```
arr = [5, 2, 8, 3, 1, 4, 6]
arr = merge_sort(arr)
print(arr)  # [1, 2, 3, 4, 5, 6, 8]
```
Let me know if you have any questions or need further clarification!

## Reflection Step

Now, let's allow the LLM to reflect on its outputs by defining another system prompt. This system prompt will tell the LLM to act as Andrej Karpathy, computer scientist and Deep Learning wizard.

>To be honest, I don't think the fact of acting like Andrej Karpathy will influence the LLM outputs, but it was fun :)

<img src="../img/karpathy.png" alt="Alt text" width="500"/>

In [10]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `mergesort_code` to the `reflection_chat_history`.

In [11]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": mergesort_code
    }
)

Now, let's generate a critique to the Python code.

In [12]:
critique = client.chat.completions.create(
    messages=reflection_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [13]:
display_markdown(critique, raw=True)

Thank you for sharing this implementation of the Merge Sort algorithm in Python. I'll provide a critique of your code and offer some recommendations for improvement.

**Code Quality and Readability:**
Your code is well-organized, and the variable names are descriptive. The docstrings are a great addition, providing a clear understanding of what each function does and what it returns. The code is easy to follow, and the logic is well-separated into two functions: `merge_sort` and `merge`.

**Algorithm Correctness:**
The Merge Sort algorithm is correctly implemented. The `merge_sort` function recursively divides the array into halves until each half has only one element, and then the `merge` function combines these sorted halves into a single sorted array.

**Performance:**
The time complexity of O(n log n) and space complexity of O(n) are correct for the Merge Sort algorithm. Your implementation has a linear space complexity because you create temporary arrays to store the left and right halves, as well as the merged result.

**Recommendations:**

1. **Type Hints:** Consider adding type hints for the function parameters and return types. This will make your code more readable and self-documenting. For example, you can add type hints for the `arr` parameter and the return type in the `merge_sort` function.

2. **Error Handling:** Your code assumes that the input `arr` is a list of integers. However, if the input is not a list or contains non-integer elements, your code will raise an error. You can add input validation to handle such cases and provide a more robust implementation.

3. **Merge Function Optimization:** In the `merge` function, you can use a single loop to merge the two sorted arrays. You can avoid the `extend` method and instead use a loop to append elements from the remaining array. This can slightly improve performance.

4. **Code Consistency:** In the `merge` function, you use both `extend` and `append` methods to add elements to the `result` array. For consistency, you can use a single approach throughout the code.

Here's an updated version of your code incorporating these recommendations:

```python
def merge_sort(arr: list[int]) -> list[int]:
    """
    ...
    """
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left = merge_sort(left)
    right = merge_sort(right)

    return merge(left, right)


def merge(left: list[int], right: list[int]) -> list[int]:
    """
    ...
    """
    result = []
    i, j = 0, 0
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1

    while i < len(left):
        result.append(left[i])
        i += 1
    while j < len(right):
        result.append(right[j])
        j += 1

    return result
```

Overall, your implementation is correct and well-organized. With these minor suggestions, your code can become even more robust and efficient.

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [14]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)

## Generation Step (II)

In [15]:
essay = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [16]:
display_markdown(essay, raw=True)

Thank you for the thorough critique and recommendations! I'm glad to hear that my implementation is correct and well-organized.

I completely agree with your suggestions:

1. **Type Hints:** Adding type hints can make the code more readable and self-documenting. I'll make sure to add them in the future.

2. **Error Handling:** Input validation is crucial in making the code more robust. I can add checks to ensure that the input is a list of integers and provide error messages for invalid inputs.

3. **Merge Function Optimization:** Your suggestion for optimizing the `merge` function is spot on. Using a single loop to merge the two sorted arrays can improve performance.

4. **Code Consistency:** I'll make sure to use a consistent approach throughout the code. In this case, using a single loop to append elements to the `result` array is a better approach.

Here's the updated code incorporating your suggestions:
```python
def merge_sort(arr: list[int]) -> list[int]:
    """
    Sorts an array of integers using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The array to be sorted
    :return: The sorted array
    """
    if not isinstance(arr, list):
        raise TypeError("Input must be a list")
    if not all(isinstance(x, int) for x in arr):
        raise ValueError("Input list must contain only integers")

    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left = merge_sort(left)
    right = merge_sort(right)

    return merge(left, right)


def merge(left: list[int], right: list[int]) -> list[int]:
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    result = []
    i, j = 0, 0
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1

    while i < len(left):
        result.append(left[i])
        i += 1
    while j < len(right):
        result.append(right[j])
        j += 1

    return result
```
Thank you again for your detailed feedback! I'll make sure to keep these suggestions in mind for future implementations.

## And the iteration starts again ...

After **Generation Step (II)** the corrected Python code will be received, once again, by Karpathy. Then, the LLM will reflect on the corrected output, suggesting further improvements and the loop will go, over and over for a number **n** of total iterations.

> There's another possibility. Suppose the Reflection step can't find any further improvement. In this case, we can tell the LLM to output some stop string, like "OK" or "Good" that means the process can be stopped. However, we are going to follow the first approach, that is, iterating for a fixed number of times.

## Implementing a class 

Now that you understand the underlying loop of the Reflection Agent, let's implement this agent as a class.

In [17]:
from agentic_patterns import ReflectionAgent

In [18]:
agent = ReflectionAgent()

In [19]:
generation_system_prompt = "You are a Python programmer tasked with generating high quality Python code"

reflection_system_prompt = "You are Andrej Karpathy, an experienced computer scientist"

user_msg = "Generate a Python implementation of the Merge Sort algorithm"

### POR ALGUM MOTIVO OCORRE ERRO ABAIXO, PORÉM NOS TESTES OCORRE PERFEITAMENTE.

In [20]:
final_response = agent.run(
    user_msg=user_msg,
    generation_system_prompt=generation_system_prompt,
    reflection_system_prompt=reflection_system_prompt,
    n_steps=10,
    verbose=1,
)


STEP 1/10



BadRequestError: Error code: 400 - {'error': {'message': 'The model `llama-3.1-70b-versatile` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}

## Final result

In [ ]:
display_markdown(final_response, raw=True)